In [1]:
!pip install ensure

In [2]:
%cd /content/drive/MyDrive/Project_Sem5
%pwd

/content/drive/MyDrive/Project_Sem5


'/content/drive/MyDrive/Project_Sem5'

In [3]:
from box import ConfigBox

# **Logger**

In [4]:
import os
import sys
import logging

logging_str = "[%(asctime)s: %(levelname)s: %(module)s: %(message)s]"

log_dir = "logs"
log_filepath = os.path.join(log_dir,"running_logs.log")
os.makedirs(log_dir, exist_ok=True)


logging.basicConfig(
    level= logging.INFO,
    format= logging_str,

    handlers=[
        logging.FileHandler(log_filepath),
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger("cnnClassifierLogger")

In [5]:
logger.info("Hello")

## **config.json Content**

In [6]:
config = ConfigBox({
    "artifact_root" : "artifacts",

    "data_ingestion" : {
        "root_dir" : "artifacts/data_ingestion",
        "source_data_dir" : "artifacts/data_ingestion/dataset",
        "data_dir" : {
            "potato" : "artifacts/data_ingestion/dataset/potato",
            "tomato" : "artifacts/data_ingestion/dataset/tomato"
        }
    },

    "prepare_base_model" : {
        "root_dir" : "artifacts/prepare_base_model",
        "model_dir" : {
            "tomato" : "artifacts/prepare_base_model/tomato",
            "potato" : "artifacts/prepare_base_model/potato"
        },
        "base_model_path" : {
            "tomato" : "artifacts/prepare_base_model/tomato/base_model.h5",
            "potato" : "artifacts/prepare_base_model/potato/base_model.h5"
        },
        "updated_model_path" : {
            "tomato" : "artifacts/prepare_base_model/tomato/updated_base_model.h5",
            "potato" : "artifacts/prepare_base_model/potato/updated_base_model.h5"
        }
    },

    "prepare_callbacks" : {
        "root_dir" : "artifacts/prepare_callbacks",
        "model_dir" : {
            "tomato" : "artifacts/prepare_callbacks/tomato",
            "potato" : "artifacts/prepare_callbacks/potato"
        },
        "tensorboard_root_log_dir" : {
            "tomato" : "artifacts/prepare_callbacks/tomato/tensorboard_log_dir",
            "potato" : "artifacts/prepare_callbacks/potato/tensorboard_log_dir"
        },
        "checkpoint_model_filepath" : {
            "tomato" : "artifacts/prepare_callbacks/tomato/checkpoint_dir/model.h5",
            "potato" : "artifacts/prepare_callbacks/potato/checkpoint_dir/model.h5"
        }
    },

    "training" : {
        "root_dir" : "artifacts/training",
        "model_dir" : {
            "potato" : "artifacts/training/potato",
            "tomato" : "artifacts/training/tomato"
        },
        "trained_model_path" : {
            "potato" : "artifacts/training/potato/model.h5",
            "tomato" : "artifacts/training/tomato/model.h5"
        }
    }
})

# **params.json Content**

In [7]:
params = ConfigBox({
    "AUGUMENTATION" : True,
    "IMAGE_SIZE" : {
        "potato" : [256, 256, 3],
        "tomato" : [224, 224, 3]
    },
    "BATCH_SIZE" : 16,
    "EPOCHS" : 30,
    "INCLUDE_TOP" : False,
    "WEIGHTS" : "imagenet",
    "CLASSES" : {
        "potato" : 3,
        "tomato" : 10
    },
    "TARGET_SIZE" : {
        "potato" : (256, 256),
        "tomato" : (224, 224)
    },
    "CLASS_MODE" : {
      "potato" : "categorical",
      "tomato" : "categorical"
    }
})

In [ ]:
params.WEIGHTS

'imagenet'

# **utils**

In [12]:
import os
import json
from ensure import ensure_annotations
from pathlib import Path

@ensure_annotations
def create_directories(path_to_directories: list, verbose=True):
    """create list of directories

    Args:
        path_to_directories (list): list of path of directories
        ignore_log (bool, optional): ignore if multiple dirs is to be created. Defaults to False.
    """
    for path in path_to_directories:
        os.makedirs(path, exist_ok=True)
        if verbose:
            print(f"created directory at: {path}")

@ensure_annotations
def save_json(path: Path, data: dict):
    """save json data

    Args:
        path (Path): path to json file
        data (dict): data to be saved in json file
    """
    with open(path, "w") as f:
        json.dump(data, f, indent=4)

    logger.info(f"json file saved at: {path}")

# **DataIngestionTrial**

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_data_dir: Path
  data_dir : ConfigBox

In [14]:
class DataIngestion:
  def __init__(self, config):
    self.config = config

# **PrepareBaseModel Trial**

In [15]:
from dataclasses import dataclass
from pathlib import Path
from box import ConfigBox

@dataclass(frozen=True)
class PrepareBaseModelConfig:
  root_dir: Path
  base_model_path: ConfigBox
  model_dir: ConfigBox
  updated_model_path: ConfigBox
  params_image_size: ConfigBox
  params_classes: ConfigBox
  params_include_top: bool
  params_weights: str

In [16]:
import tensorflow as tf
import keras
from keras.layers import Flatten, Dense
from keras.models import Model
from pathlib import Path
from box import config_box

class PrepareBaseModel:
  def __init__(self, config: PrepareBaseModelConfig):
    self.config = config

  def get_base_model(self, names) :
    self.base_model = ConfigBox()
    for name in names:
      model = keras.applications.InceptionV3(
        input_shape = self.config.params_image_size[f"{name}"],
        include_top = self.config.params_include_top,
        weights =  self.config.params_weights
      )
      self.base_model[f'{name}'] = model
      # self.save_model(path=self.config.base_model_path[f"{name}"], model=model)

  @staticmethod
  def save_model(path: Path, model: keras.Model) :
    model.save(path)

  @staticmethod
  def _prepare_base_model(model, classes) :
    for layer in model.layers:
      layer.trainable=False

    x = Flatten()(model.output)
    pred = Dense(classes, activation='softmax')(x)
    full_model = Model(inputs=model.input, outputs=pred)

    full_model.compile(
      loss='categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy']
    )

    full_model.summary()
    return full_model

  def updated_base_model(self, name: list) :
    self.updated_models = ConfigBox()
    for i in name:
      model = self._prepare_base_model(
        model = self.base_model[f"{i}"],
        classes = self.config.params_classes[f"{i}"]
      )

      self.updated_models[f"{i}"] = model
      self.save_model(path = self.config.updated_model_path[f"{i}"], model=model)

# **PrepareCallbacks Trial**

In [17]:
import time
from dataclasses import dataclass
from pathlib import Path
from box import ConfigBox

@dataclass(frozen=True)
class PrepareCallbackConfig:
  root_dir: Path
  tensorboard_root_log_dir : ConfigBox
  checkpoint_model_filepath : ConfigBox

class PrepareCallback:
  def __init__(self, config: PrepareCallbackConfig):
    self.config = config

  def _create_tb_callback(self, names: list):
    timestamp = time.strftime("%H-%M-%S--[%d-%m-%Y]")
    tensorboard_callback = ConfigBox()
    for name in names:
      tb_running_log_dir = os.path.join(
        self.config.tensorboard_root_log_dir[f"{name}"],
        f"{timestamp}"
      )
      tensorboard_callback[f"{name}"] = keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    return tensorboard_callback

  def _create_ckpt_callback(self, names: list):
    checkpoint_callback = ConfigBox()

    for name in names:
      callback = keras.callbacks.ModelCheckpoint(
          filepath=self.config.checkpoint_model_filepath[f"{name}"],
          monitor="val_accuracy",
          mode="max",
          save_best_only = True,
      )

      checkpoint_callback[f"{name}"] = callback

    return checkpoint_callback

  def get_tb_ckpt_callbacks(self, names: list) -> ConfigBox:
    tensorboard_callback = self._create_tb_callback(names)
    checkpoint_callback = self._create_ckpt_callback(names)

    a =  ConfigBox({
        "tensorboard_callback" : tensorboard_callback,
        "checkpoint_callback" : checkpoint_callback
    })

    print(a)
    return a


# **Training Trial**

In [18]:
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img

from dataclasses import dataclass
from box import ConfigBox
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
  root_dir: Path
  trained_model_path: ConfigBox
  updated_model_path: ConfigBox
  trainning_data: ConfigBox
  params_epochs: int
  params_batch_size: int
  params_target_size: ConfigBox
  params_class_mode: ConfigBox


class Training:
  def __init__(self, config: TrainingConfig) :
    self.config = config

  def get_base_model(self, names: list) :
    self.model = ConfigBox()
    for name in names:
      self.model[f"{name}"] = keras.models.load_model(
          self.config.updated_model_path[f"{name}"]
      )

  def train_valid_generator(self, names: list) :
    self.valid_set = ConfigBox()
    self.train_set = ConfigBox()
    self.test_set = ConfigBox()

    train_datagen = ImageDataGenerator(
      rescale=1./255,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
    )

    valid_datagen = ImageDataGenerator(rescale=1./255)

    test_datagen = ImageDataGenerator(rescale=1./255)

    for name in names:
      print(os.path.join(self.config.trainning_data[f"{name}"], "train"))
      print(os.path.join(self.config.trainning_data[f"{name}"], "valid"))
      print(os.path.join(self.config.trainning_data[f"{name}"], "test"))


      self.train_set[f"{name}"] = train_datagen.flow_from_directory(
          os.path.join(self.config.trainning_data[f"{name}"], "train"),
          target_size = self.config.params_target_size[f"{name}"],
          batch_size = self.config.params_batch_size,
          class_mode = self.config.params_class_mode[f"{name}"]
      )

      self.valid_set[f"{name}"] = valid_datagen.flow_from_directory(
          os.path.join(self.config.trainning_data[f"{name}"], "valid"),
          target_size = self.config.params_target_size[f"{name}"],
          batch_size = self.config.params_batch_size,
          class_mode = self.config.params_class_mode[f"{name}"]
      )

      self.test_set[f"{name}"] = test_datagen.flow_from_directory(
          os.path.join(self.config.trainning_data[f"{name}"], "test"),
          target_size = self.config.params_target_size[f"{name}"],
          batch_size = self.config.params_batch_size,
          class_mode = self.config.params_class_mode[f"{name}"]
      )

      print("name : ", name)
      print(self.train_set[f"{name}"].class_indices)

  def train(self, callbacks: ConfigBox, names: list):
    self.callbacks = ConfigBox()

    for name in names:
      self.callbacks[f"{name}"] = [callbacks.tensorboard_callback[f"{name}"], callbacks.checkpoint_callback[f"{name}"]]

    for name in names:
      self.model[f"{name}"].fit(
          self.train_set[f"{name}"],
          validation_data=self.valid_set[f"{name}"],
          epochs=self.config.params_epochs,
          steps_per_epoch=len(self.train_set[f"{name}"]),
          validation_steps=len(self.valid_set[f"{name}"]),
          callbacks=self.callbacks[f"{name}"]
      )

      self.save_model(path=self.config.trained_model_path[f"{name}"] , model=self.model[f"{name}"])

  @staticmethod
  def save_model(path: Path, model: keras.Model) :
    model.save(path)

# **Model Evaluation Trail**

In [19]:
import os
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf
from box import ConfigBox

@dataclass(frozen=True)
class ModelEvaluationConfig:
  path_of_model: ConfigBox
  data_dir: ConfigBox
  params_target_size: ConfigBox
  params_class_mode: ConfigBox
  params_batch_size: int

class ModelEvaluation:
  def __init__(self, config: ModelEvaluationConfig):
    self.config = config

  @staticmethod
  def load_model(path: Path) -> tf.keras.Model:
    return tf.keras.models.load_model(path)

  def test_generator(self, names: list) :
    self.test_set = ConfigBox()
    test_datagen = ImageDataGenerator(rescale=1./255)

    for name in names:
      self.test_set[f"{name}"] = test_datagen.flow_from_directory(
        os.path.join(self.config.data_dir[f"{name}"], "test"),
        target_size = self.config.params_target_size[f"{name}"],
        batch_size = self.config.params_batch_size,
        class_mode = self.config.params_class_mode[f"{name}"]
      )

  def evaluation(self, names: list) :
    self.test_generator(names)
    self.model = ConfigBox()
    self.score = ConfigBox()
    for name in names:
      model = self.load_model(self.config.path_of_model[f"{name}"])
      score = model.evaluate(self.test_set[f"{name}"])
      self.score[f"{name}"] = {"loss": score[0], "accuracy": score[1]}
      self.model[f"{name}"] = model

    print(self.score)
    save_json(path=Path("scores.json"), data=self.score)



# **config**

In [20]:
import os
class ConfigurationManager:
  def __init__(self, config, params):
    self.config = config
    self.params = params

    create_directories([self.config.artifact_root])

  def get_data_ingestion_config(self) -> DataIngestionConfig:
    curr_config = self.config.data_ingestion
    create_directories([curr_config.root_dir])

    data_ingestion_config = DataIngestionConfig(
        root_dir = curr_config.root_dir,
        source_data_dir = curr_config.source_data_dir,
        data_dir = curr_config.data_dir
    )

    return data_ingestion_config

  def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
    curr_config = self.config.prepare_base_model
    curr_params = self.params
    create_directories([curr_config.root_dir])

    prepare_base_model_config = PrepareBaseModelConfig(
      root_dir = Path(curr_config.root_dir),
      model_dir = curr_config.model_dir,
      base_model_path = curr_config.base_model_path,
      updated_model_path = curr_config.updated_model_path,
      params_include_top = curr_params.INCLUDE_TOP,
      params_image_size = curr_params.IMAGE_SIZE,
      params_weights = curr_params.WEIGHTS,
      params_classes = curr_params.CLASSES
    )

    return prepare_base_model_config

  def get_prepare_callback_config(self) -> PrepareCallbackConfig:
    curr_config = self.config.prepare_callbacks
    model_ckpt_dir = []
    for model in curr_config.checkpoint_model_filepath:
      dir = curr_config.checkpoint_model_filepath[f"{model}"]
      dir = os.path.dirname(dir)
      model_ckpt_dir.append(dir)

    model_tb_dir = []
    for model in curr_config.tensorboard_root_log_dir:
      dir = curr_config.tensorboard_root_log_dir[f"{model}"]
      model_tb_dir.append(dir)

    create_directories(model_ckpt_dir)
    create_directories(model_tb_dir)

    prepare_callback_dir = PrepareCallbackConfig (
        root_dir = curr_config.root_dir,
        tensorboard_root_log_dir = curr_config.tensorboard_root_log_dir,
        checkpoint_model_filepath = curr_config.checkpoint_model_filepath
    )

    return prepare_callback_dir

  def get_training_config(self) -> TrainingConfig:
    training = self.config.training
    params = self.params
    create_directories([training.root_dir])

    training = TrainingConfig(
      root_dir = training.root_dir,
      trained_model_path = training.trained_model_path,
      updated_model_path = self.config.prepare_base_model.updated_model_path,
      trainning_data = self.config.data_ingestion.data_dir,
      params_epochs = params.EPOCHS,
      params_batch_size = params.BATCH_SIZE,
      params_target_size = params.TARGET_SIZE,
      params_class_mode = params.CLASS_MODE
    )

    return training

  def get_model_evaluation_config(self) -> ModelEvaluationConfig:
    model_evaluation = ModelEvaluationConfig(
        path_of_model = self.config.prepare_callbacks.checkpoint_model_filepath,
        data_dir = self.config.data_ingestion.data_dir,
        params_target_size = self.params.TARGET_SIZE,
        params_batch_size = self.params.BATCH_SIZE,
        params_class_mode = self.params.CLASS_MODE
    )

    return model_evaluation

# **Pipeline**

In [ ]:
## DataIngestion Pipeline

try:
  config1 = ConfigurationManager(config, params)
  data_ingestion_config = config1.get_data_ingestion_config()
  data_ingestion = DataIngestion(config=data_ingestion_config)
except Exception as e:
  raise e

In [ ]:
## PrepareBaseModel Pipeline

try:
  config1 = ConfigurationManager(config, params)
  prepare_base_model_config = config1.get_prepare_base_model_config()
  prepare_base_model = PrepareBaseModel(config = prepare_base_model_config)
  prepare_base_model.get_base_model(['potato', 'tomato'])
  prepare_base_model.updated_base_model(['potato', 'tomato'])
except Exception as e:
  raise e

In [65]:
## PrepareCallback Pipeline

try:
  config1 = ConfigurationManager(config, params)
  prepare_callback_config = config1.get_prepare_callback_config()
  prepare_callback = PrepareCallback(config = prepare_callback_config)
  callbacks = prepare_callback.get_tb_ckpt_callbacks(['potato', 'tomato'])
except Exception as e:
  raise e

created directory at: artifacts
created directory at: artifacts/prepare_callbacks/tomato/checkpoint_dir
created directory at: artifacts/prepare_callbacks/potato/checkpoint_dir
created directory at: artifacts/prepare_callbacks/tomato/tensorboard_log_dir
created directory at: artifacts/prepare_callbacks/potato/tensorboard_log_dir
{'tensorboard_callback': {'potato': <keras.src.callbacks.TensorBoard object at 0x77fe07795930>, 'tomato': <keras.src.callbacks.TensorBoard object at 0x77fe077957b0>}, 'checkpoint_callback': {'potato': <keras.src.callbacks.ModelCheckpoint object at 0x77fe077949a0>, 'tomato': <keras.src.callbacks.ModelCheckpoint object at 0x77fe07794250>}}


In [67]:
## training Pipeline

try:
  config1 = ConfigurationManager(config, params)
  training_config = config1.get_training_config()
  training = Training(config=training_config)
  training.get_base_model(['potato', 'tomato'])
  training.train_valid_generator(['potato', 'tomato'])
  training.train(
    callbacks = callbacks,
    names = ['potato', 'tomato']
  )
except Exception as e:
  raise e

created directory at: artifacts
created directory at: artifacts/training
artifacts/data_ingestion/dataset/potato/train
artifacts/data_ingestion/dataset/potato/valid
artifacts/data_ingestion/dataset/potato/test
Found 1720 images belonging to 3 classes.
Found 215 images belonging to 3 classes.
Found 214 images belonging to 3 classes.
artifacts/data_ingestion/dataset/tomato/train
artifacts/data_ingestion/dataset/tomato/valid
artifacts/data_ingestion/dataset/tomato/test
Found 4585 images belonging to 10 classes.
Found 416 images belonging to 10 classes.
Found 402 images belonging to 10 classes.
Epoch 1/30
108/108 [==============================] - ETA: 0s - loss: 1.6024 - accuracy: 0.8302

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


108/108 [==============================] - 46s 405ms/step - loss: 1.6024 - accuracy: 0.8302 - val_loss: 1.2759 - val_accuracy: 0.8879
Epoch 2/30
108/108 [==============================] - 32s 300ms/step - loss: 0.6433 - accuracy: 0.9227 - val_loss: 1.7966 - val_accuracy: 0.8879
Epoch 3/30
108/108 [==============================] - 35s 324ms/step - loss: 0.9640 - accuracy: 0.9134 - val_loss: 0.4266 - val_accuracy: 0.9439
Epoch 4/30
108/108 [==============================] - 33s 301ms/step - loss: 0.5647 - accuracy: 0.9459 - val_loss: 0.5638 - val_accuracy: 0.9393
Epoch 5/30
108/108 [==============================] - 34s 317ms/step - loss: 0.3582 - accuracy: 0.9645 - val_loss: 1.0839 - val_accuracy: 0.9159
Epoch 6/30
108/108 [==============================] - 32s 298ms/step - loss: 0.4883 - accuracy: 0.9558 - val_loss: 0.8337 - val_accuracy: 0.9346
Epoch 7/30
108/108 [==============================] - 32s 297ms/step - loss: 0.4421 - accuracy: 0.9640 - val_loss: 0.9359 - val_accuracy: 0.9

In [83]:
try:
    config1 = ConfigurationManager(config, params)
    val_config = config1.get_model_evaluation_config()
    evaluation = ModelEvaluation(config=val_config)
    evaluation.evaluation(['potato', 'tomato'])
except Exception as e:
   raise e

created directory at: artifacts
Found 214 images belonging to 3 classes.
Found 402 images belonging to 10 classes.
26/26 [==============================] - 3s 52ms/step - loss: 3.9958 - accuracy: 0.8632
{'potato': {'loss': 0.6887072324752808, 'accuracy': 0.9766355156898499}, 'tomato': {'loss': 3.9958200454711914, 'accuracy': 0.8631840944290161}}


In [21]:
try:
  config1 = ConfigurationManager(config, params)
  training_config = config1.get_training_config()
  training = Training(config=training_config)
  training.get_base_model(['potato', 'tomato'])
  training.train_valid_generator(['potato', 'tomato'])
except Exception as e:
  raise e

created directory at: artifacts
created directory at: artifacts/training
artifacts/data_ingestion/dataset/potato/train
artifacts/data_ingestion/dataset/potato/valid
artifacts/data_ingestion/dataset/potato/test
Found 1720 images belonging to 3 classes.
Found 215 images belonging to 3 classes.
Found 214 images belonging to 3 classes.
name :  potato
{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}
artifacts/data_ingestion/dataset/tomato/train
artifacts/data_ingestion/dataset/tomato/valid
artifacts/data_ingestion/dataset/tomato/test
Found 4585 images belonging to 10 classes.
Found 416 images belonging to 10 classes.
Found 402 images belonging to 10 classes.
name :  tomato
{'Tomato___Bacterial_spot': 0, 'Tomato___Early_blight': 1, 'Tomato___Late_blight': 2, 'Tomato___Leaf_Mold': 3, 'Tomato___Septoria_leaf_spot': 4, 'Tomato___Spider_mites Two-spotted_spider_mite': 5, 'Tomato___Target_Spot': 6, 'Tomato___Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato___Tomato_mosaic_